# Dependencies and Configs

In [3]:
pip install torch torchvision pyyaml

DEPRECATION: Loading egg at /opt/anaconda3/lib/python3.11/site-packages/selenium-3.141.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /opt/anaconda3/lib/python3.11/site-packages/urllib3-1.26.6-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /opt/anaconda3/lib/python3.11/site-packages/udemyscraper-0.8.2-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /opt/anaconda3/lib/python3.11/site-packages/soupsieve-2.2.1-py3.11.egg is deprecated. pip 24.3 w

**Loading Configurations from the YAML file. [You may modify the file as your favour.]**

In [18]:
import yaml
from pathlib import Path

config_path = Path().resolve().parent / 'config.dev.yaml'

with open(config_path, 'r') as config_file:
    config = yaml.safe_load(config_file)

# Preparing Train and Validation Datasets + Pre Processing

In [14]:
import torch
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms

**Define Transforms**

In [9]:
import random

In [34]:
augmentation_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
    transforms.RandomRotation(degrees=random.choice([30, 60, 90, 120, 150, 180, 210, 240, 270, 300, 330, 360])),  # Randomly choose from the specified angles
    transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.2),  # Randomly change the brightness, contrast, saturation, and hue
])

load_transofrms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

**Loading the Dataset**

In [35]:
data_directory = Path().resolve().parent / config['data_directory']
dataset = datasets.ImageFolder(root=data_directory, transform=load_transofrms)

**Spiliting into the Reproducible Train, Validation, and Test Datasets**

In [36]:
import numpy as np
from sklearn.model_selection import train_test_split
from torch.utils.data import Subset


seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)



train_and_val_share = 0.85  # 85% for train + validation, 15% for test
train_share = 0.85 # 85% for train, 15% for validation

# Further split train_val_indices into training and validation
train_and_val_indices, test_indices = train_test_split(
    list(range(len(dataset))), 
    test_size=1 - train_and_val_share, 
    random_state=seed, 
    shuffle=True
)

train_indices, val_indices = train_test_split(
    train_and_val_indices,
    test_size=1 - train_share,
    random_state=seed,
    shuffle=True
)

train_dataset = Subset(dataset, train_indices)
val_dataset = Subset(dataset, val_indices)
test_dataset = Subset(dataset, test_indices)

print(f"Train dataset size: {len(train_dataset)}")
print(f"Validation dataset size: {len(val_dataset)}")
print(f"Test dataset size: {len(test_dataset)}")

Train dataset size: 3900
Validation dataset size: 689
Test dataset size: 811


**Applying Data Augmentation Transforms on the Train Dataset**

In [37]:
train_dataset.dataset.transform = augmentation_transforms

**Defining Train, Validation, and Test Dataloaders**

In [38]:
batch_size = config['batch_size']
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
validation_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [39]:
id2label = {v: k for k, v in dataset.class_to_idx.items()}
label2id = dataset.class_to_idx
print(id2label)

{0: 'antelope', 1: 'badger', 2: 'bat', 3: 'bear', 4: 'bee', 5: 'beetle', 6: 'bison', 7: 'boar', 8: 'butterfly', 9: 'cat', 10: 'caterpillar', 11: 'chimpanzee', 12: 'cockroach', 13: 'cow', 14: 'coyote', 15: 'crab', 16: 'crow', 17: 'deer', 18: 'dog', 19: 'dolphin', 20: 'donkey', 21: 'dragonfly', 22: 'duck', 23: 'eagle', 24: 'elephant', 25: 'flamingo', 26: 'fly', 27: 'fox', 28: 'goat', 29: 'goldfish', 30: 'goose', 31: 'gorilla', 32: 'grasshopper', 33: 'hamster', 34: 'hare', 35: 'hedgehog', 36: 'hippopotamus', 37: 'hornbill', 38: 'horse', 39: 'hummingbird', 40: 'hyena', 41: 'jellyfish', 42: 'kangaroo', 43: 'koala', 44: 'ladybugs', 45: 'leopard', 46: 'lion', 47: 'lizard', 48: 'lobster', 49: 'mosquito', 50: 'moth', 51: 'mouse', 52: 'octopus', 53: 'okapi', 54: 'orangutan', 55: 'otter', 56: 'owl', 57: 'ox', 58: 'oyster', 59: 'panda', 60: 'parrot', 61: 'pelecaniformes', 62: 'penguin', 63: 'pig', 64: 'pigeon', 65: 'porcupine', 66: 'possum', 67: 'raccoon', 68: 'rat', 69: 'reindeer', 70: 'rhinocero

# Training Model

**Load the pretrained ViT model**

In [40]:
from timm import create_model
pre_trained_model = config['pre_trained_model']
model = create_model(pre_trained_model, pretrained=True, num_classes=len(train_dataset.dataset.classes))

**Moving model to GPU if available**

In [41]:
if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("MPS is available and will be used.")
else:
    device = torch.device("cpu")
    print("MPS is not available, using CPU.")

MPS is available and will be used.


**Defining loss function and optimizer**

In [42]:
import torch.nn as nn
import torch.optim as optim

In [43]:
criterion = nn.CrossEntropyLoss()  # For classification
optimizer = optim.Adam(model.parameters(), lr=1e-4)

**Implementing the Early Stop Method**

In [44]:
def early_stopping(val_loss, best_loss, patience_counter, patience, min_delta):
    if best_loss is None or val_loss < best_loss - min_delta:
        return val_loss, 0
    else:
        return best_loss, patience_counter + 1  # Increment counter if no improvement

**The Training Loop**

In [45]:
from tqdm import tqdm  # For progress bar

In [46]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
model.to(device)
print(device)

mps


In [47]:
train_loader

In [48]:
num_epochs = config['num_epochs']
patience = config['patience']
learning_rate = config['learning_rate']

best_loss = None
patience_counter = 0

for epoch in range(num_epochs):
    model.train()  # Set model to training mode
    running_loss = 0.0

    # Train over batches
    for images, labels in tqdm(train_loader):
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()  # Clear previous gradients
        loss.backward()  # Compute gradients
        optimizer.step()  # Update weights

        running_loss += loss.item()

    # Print epoch loss
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader)}')

    # Validation Loop
    model.eval()  # Set model to evaluation mode
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():  # Disable gradient calculation for validation
        for images, labels in validation_loader:
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)  # Compute validation loss
            val_loss += loss.item()  # Accumulate validation loss

            # Calculate accuracy
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    # Calculate average validation loss
    avg_val_loss = val_loss / len(validation_loader)

    # Print validation metrics
    print(f'Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {100 * correct / total:.2f}%')

    # Early stopping check
    best_loss, patience_counter = early_stopping(avg_val_loss, best_loss, patience_counter, patience, learning_rate)

    if patience_counter >= patience:
        print("Early stopping triggered!")
        break

# Test Loop
model.eval()  # Set the model to evaluation mode
test_loss = 0.0
correct = 0
total = 0
with torch.no_grad():  # Disable gradient calculation for testing
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)

        # Update metrics
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# Print test metrics
test_accuracy = 100 * correct / total
print(f'Test Accuracy: {test_accuracy:.2f}%')

print("Training completed!")

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 122/122 [02:05<00:00,  1.03s/it]


Epoch [1/50], Loss: 3.137543565914279
Validation Loss: 1.8834, Validation Accuracy: 54.86%


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 122/122 [01:47<00:00,  1.13it/s]


Epoch [2/50], Loss: 1.2509175837039948
Validation Loss: 1.5061, Validation Accuracy: 63.28%


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 122/122 [01:41<00:00,  1.20it/s]


...

Epoch [9/50], Loss: 0.2040114451932614
Validation Loss: 1.3935, Validation Accuracy: 69.23%


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 122/122 [01:40<00:00,  1.21it/s]


Epoch [10/50], Loss: 0.17616930097097255
Validation Loss: 1.3894, Validation Accuracy: 70.68%


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 122/122 [01:40<00:00,  1.21it/s]


Epoch [11/50], Loss: 0.17478094863720606
Validation Loss: 1.3309, Validation Accuracy: 69.81%
Early stopping triggered!
Test Accuracy: 71.02%
Training completed!


**Saving the Model**

In [1]:
# Save the trained model to a file
model_name = f'trained_{config["pre_trained_model"]}.pth'
model_path = Path().resolve().parent / 'models' / model_name
torch.save(model.state_dict(), model_path)
print(f"Model saved at {model_path}")

NameError: name 'config' is not defined